In [20]:
from web3 import Web3
from web3.gas_strategies.rpc import rpc_gas_price_strategy
import requests
from dotenv import load_dotenv
import os
import json
import numpy as np

load_dotenv()

class TokenTrader:
    def __init__(self, rpc_url, etherscan_api_key, wallet_address, private_key, uniswap_contract_address, token_input_address, token_output_address):
        self.rpc_url = rpc_url
        self.web3 = Web3(Web3.HTTPProvider(rpc_url))
        self.web3.eth.set_gas_price_strategy(rpc_gas_price_strategy)
        self.etherscan_api_key = etherscan_api_key
        self.wallet_address = wallet_address
        self.private_key = private_key
        # Init Uniswap objects
        self.uniswap_contract_address = self.web3.to_checksum_address(uniswap_contract_address)
        self.uniswap_swap_contract_abi = self.get_contract_abi(self.uniswap_contract_address)
        self.uniswap_swap_contract_object = self.web3.eth.contract(address=self.uniswap_contract_address, abi=self.uniswap_swap_contract_abi)
        # Init Token INPUT objects
        self.token_input_address = self.web3.to_checksum_address(token_input_address)
        self.token_input_abi = self.get_contract_abi(self.token_input_address)
        self.token_input_object = self.web3.eth.contract(address=self.token_input_address, abi=self.token_input_abi)
        self.token_input_symbol = self.token_input_object.functions.symbol().call()
        # Init Token OUTPUT objects
        self.token_output_address = self.web3.to_checksum_address(token_output_address)
        self.token_output_abi = self.get_contract_abi(self.token_output_address)
        self.token_output_object = self.web3.eth.contract(address=self.token_output_address, abi=self.token_output_abi)
        self.token_output_symbol = self.token_output_object.functions.symbol().call()

    def is_connected(self):
        return self.web3.is_connected()
     
    def get_contract_abi(self, contract_address):
        try:
            url = f"https://api.etherscan.io/api?module=contract&action=getabi&address={contract_address}&apikey={self.etherscan_api_key}"
            response = requests.get(url)
            response_json = response.json()
            # Comprueba si la solicitud fue exitosa y si hay un ABI disponible
            if response_json['status'] == '1' and response_json['message'] == 'OK':
                abi = json.loads(response_json['result'])
                return abi
        except Exception as e:
            print(f"Ha ocurrido un error al obtener el ABI: {e}")
            return None
    def get_price(self):
        # Obtiene el estado de slot0 para acceder al precio
        slot0 = self.uniswap_swap_contract_object.functions.slot0().call()
        sqrtPriceX96 = slot0[0]
        # Calcular el precio de 1 token input en términos de token output
        price_raw = (sqrtPriceX96 ** 2) / (2 ** 192)
        # Ajustar por la diferencia de decimales entre RETIK y USDT
        token_input_decimals = self.token_input_object.functions.decimals().call()
        token_output_decimals = self.token_output_object.functions.decimals().call()
        decimal_adjustment = 10 ** (token_input_decimals - token_output_decimals)
        price = price_raw * decimal_adjustment
        return sqrtPriceX96, price
    
    def swap(self, qty, sqrtPriceX96):
        # Datos del swap
        recipient_address = self.wallet_address
        zeroForOne = True  # True si RETIK es token0, False si es token1
        amountSpecified = self.web3.to_wei(qty, 'ether')
        sqrtPriceLimitX96 = int(sqrtPriceX96 * np.sqrt(1 - 0.05))
        data = b''

        # Obtener los datos codificados de la función `swap`
        swap_data = self.uniswap_swap_contract_object.encodeABI(
            fn_name='swap',
            args=[recipient_address, zeroForOne, amountSpecified, sqrtPriceLimitX96, data]
        )

        # Construir la transacción
        tx = {
            'chainId': self.web3.eth.chain_id,
            'to': self.uniswap_swap_contract_object.address,
            'data': swap_data,
            'nonce': self.web3.eth.get_transaction_count(self.wallet_address),
            'gas': 2000000,
            'gasPrice': self.web3.eth.generate_gas_price()
        }
        # Firmar y enviar la transacción
        signed_tx = self.web3.eth.account.sign_transaction(tx, self.private_key)
        tx_hash = self.web3.eth.send_raw_transaction(signed_tx.rawTransaction)
        return tx_hash

# Configuración del proveedor (Infura)
rpc_url = "https://rpc.ankr.com/eth"
etherscan_api_key = os.getenv("ETHERSCAN_API_KEY")
wallet_address = os.getenv("WALLET_ADDRESS")
private_key = os.getenv("PRIVATE_KEY")
uniswap_contract_address = "0x1458770554b8918B970444d8b2c02A47F6dF99A7"
token_input_address = "0x26EbB8213fb8D66156F1Af8908d43f7e3e367C1d"
token_output_address = "0xdAC17F958D2ee523a2206206994597C13D831ec7"
token_trader = TokenTrader(rpc_url,
                           etherscan_api_key,
                           wallet_address, 
                           private_key,
                           uniswap_contract_address,
                           token_input_address,
                           token_output_address)
# Está hecha la conexión con Ethereum
if token_trader.is_connected():
    print("Conectado a Ethereum")
else:
    print("Error al conectar a Ethereum")
sqrtPriceX96, price = token_trader.get_price()
print(f"sqrtPriceX96: {sqrtPriceX96} = {price}")
print(f"{token_trader.swap(1, sqrtPriceX96).hex()}")

Conectado a Ethereum
sqrtPriceX96: 115840752863287831483948 = 2.137782784096726
Transaction Hash: 0x539c5e0003bb4d625c1c520ea9cee674bcab66b4084f778de09e4ce2ebf0d2c4
b'S\x9c^\x00\x03\xbbMb\\\x1cR\x0e\xa9\xce\xe6t\xbc\xabf\xb4\x08Ow\x8d\xe0\x9eL\xe2\xeb\xf0\xd2\xc4'
